# Parse Phenobooth data

Notebook for parsing raw data from phenobooth and computing averages/stds for each gene/condition.

2019-12-19

## Initial bolierplate

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from os.path import join, dirname, basename, exists, isdir

### Load environmental variables from the project root directory ###
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

# now you can get the variables using their names

# Check whether a network drive has been specified
DATABASE = os.environ.get("NETWORK_URL")
if DATABASE == 'None':
    pass
else:
    pass
    #mount network drive here

# set up directory paths
CURRENT_DIR = os.getcwd()
PROJ = dirname(dotenv_path) # project root directory

DATA = join(PROJ, 'data') #data directory
RAW_EXTERNAL = join(DATA, 'raw_external') # external data raw directory
RAW_INTERNAL = join(DATA, 'raw_internal') # internal data raw directory
INTERMEDIATE = join(DATA, 'intermediate') # intermediate data directory
FINAL = join(DATA, 'final') # final data directory

RESULTS = join(PROJ, 'results') # output directory
FIGURES = join(RESULTS, 'figures') # figure output directory
PICTURES = join(RESULTS, 'pictures') # picture output directory


# make folders specific for certain data
folder_name = ''
if folder_name != '':
    #make folders if they don't exist
    if not exists(join(RAW_EXTERNAL, folder_name)):
        os.makedirs(join(RAW_EXTERNAL, folder_name))

    if not exists(join(INTERMEDIATE, folder_name)):
        os.makedirs(join(INTERMEDIATE, folder_name))

    if not exists(join(FINAL, folder_name)):
        os.makedirs(join(FINAL, folder_name))

print('Standard variables loaded, you are good to go!')

Standard variables loaded, you are good to go!


## 1. Load data

We will use:
* The original raw data with the fluorescence measurements of each 1536-well plate.
* A lookup table connecting positions in the 384-well plate to gene ids.

In [2]:
import pandas as pd

data = pd.read_csv(join(RAW_INTERNAL,"Betax05.12.19_ColonyData_AllPlates.txt"), sep="\t")
print(data)

       Run  Plate     Type Row  Col  Size  Circularity  Brightness  Redness  \
0        2     14  Control   A    1     0       0.0000    0.000000      0.0   
1        2     14  Control   A    2     0       0.0000    0.000000      0.0   
2        2     14  Control   A    3   528       0.9261   20.986111    -52.2   
3        2     14  Control   A    4   411       0.9294   19.626115    -48.9   
4        2     14  Control   A    5   509       0.9489   19.151277    -47.7   
...    ...    ...      ...  ..  ...   ...          ...         ...      ...   
39931    3     13  Control  AF   44     0       0.0000    0.000000      0.0   
39932    3     13  Control  AF   45   537       0.9572  137.873371      8.8   
39933    3     13  Control  AF   46   513       0.9532  130.765432     10.9   
39934    3     13  Control  AF   47   431       0.9629  129.105955     11.3   
39935    3     13  Control  AF   48   551       0.9673  129.872958     10.9   

       Avg. Red  Avg. Blue  Avg. Green  Multi  Gene

In [3]:
gene_ids = pd.read_csv(join(RAW_EXTERNAL,"geneIDs.txt"), sep="\t")
print(gene_ids)

      Plate # Row  Column        ORF       Gene  \
0           1   A       1      Blank      Blank   
1           1   A       2    YLL040C      VPS13   
2           1   A       3    YAL068C       PAU8   
3           1   A       4      Blank      Blank   
4           1   A       5    YAL067C       SEO1   
...       ...  ..     ...        ...        ...   
4987       13   P      20      Blank      Blank   
4988       13   P      21    YPR072W       NOT5   
4989       13   P      22      Blank      Blank   
4990       13   P      23  YOR008C-A  YOR008C-A   
4991       13   P      24      Blank      Blank   

                                             Decription  \
0                                                 Blank   
1     Protein of unknown function; heterooligomeric ...   
2     Protein of unknown function, member of the ser...   
3                                                 Blank   
4     Putative permease, member of the allantoate tr...   
...                              

## 2. Parse data location

Now that we have the data, the first step is to translate the positions in the 1536-well plate to the more standard 384-well plate. For this, we have to consider that the 1536-well plate has each colony from the 384-well plate in 4 replicates, using the following order:

| | Column 1 | Column 2 | Column 3 | Column 4 | ... |
|:-----:|:----:|:----:|:----:|:----:|:---:|
| Row A | _A1_ | _A1_ | _A2_ | _A2_ | ... |
| Row B | _A1_ | _A1_ | _A2_ | _A2_ | ... |
| Row C | _B1_ | _B1_ | _B2_ | _B2_ | ... |
| Row D | _B1_ | _B1_ | _B2_ | _B2_ | ... |
| ... | ... | ... | ... | ... | ... |

So we will create a new variable called `mapped.pos` that stores the original position in the 384-well plates:

In [4]:
import math

# We need an array of all letters so that we can use the position in the array as
# the number representing each letter. First position is empty as it is zero:
letters = ["-", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N","O", "P",
           "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "AC", "AD", "AE", "AF"]

# Iterate through the dataset and find the original position:
data["mapped.pos"] = ""
for index, row in data.iterrows():
    if row["Row"] == "A":
        letter_number = 1
        current_letter = "A"
    elif row["Row"] != current_letter:
        letter_number += 1
        current_letter = row["Row"]
    # Both the column and row should be halved and rounded up (to account for the replicates):
    letter = letters[math.ceil(letter_number/2)]
    number = math.ceil(row["Col"]/2)
    data.iloc[index,-1] = letter + str(number)
print(data)

       Run  Plate     Type Row  Col  Size  Circularity  Brightness  Redness  \
0        2     14  Control   A    1     0       0.0000    0.000000      0.0   
1        2     14  Control   A    2     0       0.0000    0.000000      0.0   
2        2     14  Control   A    3   528       0.9261   20.986111    -52.2   
3        2     14  Control   A    4   411       0.9294   19.626115    -48.9   
4        2     14  Control   A    5   509       0.9489   19.151277    -47.7   
...    ...    ...      ...  ..  ...   ...          ...         ...      ...   
39931    3     13  Control  AF   44     0       0.0000    0.000000      0.0   
39932    3     13  Control  AF   45   537       0.9572  137.873371      8.8   
39933    3     13  Control  AF   46   513       0.9532  130.765432     10.9   
39934    3     13  Control  AF   47   431       0.9629  129.105955     11.3   
39935    3     13  Control  AF   48   551       0.9673  129.872958     10.9   

       Avg. Red  Avg. Blue  Avg. Green  Multi  Gene

Also, and specifically in this dataset, the plate number for the second run should be reduced by 13:

In [5]:
for index, row in data.iterrows():
    if row["Run"] == 2:
        data.iloc[index,data.columns.get_loc("Plate")] -= 13
print(data)

       Run  Plate     Type Row  Col  Size  Circularity  Brightness  Redness  \
0        2      1  Control   A    1     0       0.0000    0.000000      0.0   
1        2      1  Control   A    2     0       0.0000    0.000000      0.0   
2        2      1  Control   A    3   528       0.9261   20.986111    -52.2   
3        2      1  Control   A    4   411       0.9294   19.626115    -48.9   
4        2      1  Control   A    5   509       0.9489   19.151277    -47.7   
...    ...    ...      ...  ..  ...   ...          ...         ...      ...   
39931    3     13  Control  AF   44     0       0.0000    0.000000      0.0   
39932    3     13  Control  AF   45   537       0.9572  137.873371      8.8   
39933    3     13  Control  AF   46   513       0.9532  130.765432     10.9   
39934    3     13  Control  AF   47   431       0.9629  129.105955     11.3   
39935    3     13  Control  AF   48   551       0.9673  129.872958     10.9   

       Avg. Red  Avg. Blue  Avg. Green  Multi  Gene

## 3. Add gene information

The gene information already comes in the 384-well plate standard, so we can recreate the mapped position by just combining `Row` and `Column`:

In [6]:
gene_ids["mapped.pos"] = gene_ids["Row"] + gene_ids["Column"].astype('str')
gene_ids = gene_ids.rename(columns={'Plate #':'Plate'})
print(gene_ids)

      Plate Row  Column        ORF       Gene  \
0         1   A       1      Blank      Blank   
1         1   A       2    YLL040C      VPS13   
2         1   A       3    YAL068C       PAU8   
3         1   A       4      Blank      Blank   
4         1   A       5    YAL067C       SEO1   
...     ...  ..     ...        ...        ...   
4987     13   P      20      Blank      Blank   
4988     13   P      21    YPR072W       NOT5   
4989     13   P      22      Blank      Blank   
4990     13   P      23  YOR008C-A  YOR008C-A   
4991     13   P      24      Blank      Blank   

                                             Decription  \
0                                                 Blank   
1     Protein of unknown function; heterooligomeric ...   
2     Protein of unknown function, member of the ser...   
3                                                 Blank   
4     Putative permease, member of the allantoate tr...   
...                                                 ...  

Now we can merge both datasets, by combining any row that shares the plate number and the mapped position:

In [7]:
data = pd.merge(data, gene_ids, how='left', on=['Plate', 'mapped.pos'])
print(data)

       Run  Plate     Type Row_x  Col  Size  Circularity  Brightness  Redness  \
0        2      1  Control     A    1     0       0.0000    0.000000      0.0   
1        2      1  Control     A    2     0       0.0000    0.000000      0.0   
2        2      1  Control     A    3   528       0.9261   20.986111    -52.2   
3        2      1  Control     A    4   411       0.9294   19.626115    -48.9   
4        2      1  Control     A    5   509       0.9489   19.151277    -47.7   
...    ...    ...      ...   ...  ...   ...          ...         ...      ...   
39931    3     13  Control    AF   44     0       0.0000    0.000000      0.0   
39932    3     13  Control    AF   45   537       0.9572  137.873371      8.8   
39933    3     13  Control    AF   46   513       0.9532  130.765432     10.9   
39934    3     13  Control    AF   47   431       0.9629  129.105955     11.3   
39935    3     13  Control    AF   48   551       0.9673  129.872958     10.9   

       Avg. Red  ...  mappe

## 4. Shape intermediate data

A copy of the complete merged dataset will be stored as intermediate data for reference. This will include:
* Key variables normalized to size
* Columns with clearer names
* Unused/redundant columns removed
* More practical order

In [8]:
# Normalize key variables:
data["Norm.Brightness"] = data["Brightness"] / data["Size"]
data["Norm.Avg. Red"] = data["Avg. Red"] / data["Size"]
data["Norm.Redness"] = data["Redness"] / data["Size"]

# Rename some column names:
data = data.rename(columns={"Gene_y":"Gene", "Row_x":"Original.Row", "Col":"Original.Col"})

# Remove unused/redundant data:
data = data.drop(columns=["Type", "Multi", "Gene_x", "Row_y", "Column"])

# Re-order dataset:            
data = data[["Run", "Plate", "Original.Row", "Original.Col", "mapped.pos", "96-position", "Gene",
             "ORF", "Decription", "GO Biological Process", "GO Molecular Function", "GO Cellular Component",
             "Size", "Circularity", "Brightness", "Norm.Brightness", "Redness", "Norm.Redness",
             "Avg. Red", "Norm.Avg. Red", "Avg. Blue", "Avg. Green"]]
print(data)

       Run  Plate Original.Row  Original.Col mapped.pos 96-position  \
0        2      1            A             1         A1        1-A1   
1        2      1            A             2         A1        1-A1   
2        2      1            A             3         A2        2-A1   
3        2      1            A             4         A2        2-A1   
4        2      1            A             5         A3        1-A2   
...    ...    ...          ...           ...        ...         ...   
39931    3     13           AF            44        P22      51-H11   
39932    3     13           AF            45        P23      70-H12   
39933    3     13           AF            46        P23      70-H12   
39934    3     13           AF            47        P24      51-H12   
39935    3     13           AF            48        P24      51-H12   

            Gene        ORF  \
0          Blank      Blank   
1          Blank      Blank   
2          VPS13    YLL040C   
3          VPS13    YLL

In [9]:
data.to_csv(join(INTERMEDIATE, "Betax05.12.19_ColonyData_merged.csv"))

## 5. Filter & summarize replicate info

The final dataset will have:
* Only the variables of interest (`Brightness`, `Avg. Red.` and `Redness`), both in their raw and normalized versions, together with the minimal descriptors for identifying the measurement (`Run`, `Plate`, `mapped.pos` and `Gene`). Note that `Gene` is redundant with `mapped.pos`, but we will keep both as `Gene` is more useful for any post-analysis.
* All blank data (used for reference in the plates) filtered out
* All zero measurements taken out, so that they don't interfere with any calculation later

In [10]:
# Filter relevant info:
data = data[["Run", "Plate", "mapped.pos", "Gene", "Size", "Brightness", "Avg. Red", "Redness",
             "Norm.Brightness", "Norm.Avg. Red", "Norm.Redness"]]

# Filter out blank data:
data = data[data.Gene != "Blank"]

# Replace any value=0 (no growth) with NaN for proper computations:
data[data == 0] = math.nan

Now we can compute for each group of 4 replicates:

* Mean values
* Standard deviations
* Count the number of replicates that grew

For achieving this, we will:

* Group all data by their identifiers to create one additional dataset for each computation.
* For mean values and standard deviations, rename each column adding a `.mean` or `.std`, respectively.
* For count, only look at the counts of positive growth according to `Size`.
* Merge all 3 datasets in the final dataset.

In [11]:
# Compute operations:
mean_data = data.groupby(["Run", "Plate", "mapped.pos", "Gene"]).mean()
std_data = data.groupby(["Run", "Plate", "mapped.pos", "Gene"]).std()
count_data = data.groupby(["Run", "Plate", "mapped.pos", "Gene"]).count()

# Rename columns:
def change_col_names(data, names, operation):
    for name in names:
        data = data.rename(columns={name:(name + "." + operation)})
    return data

names = ["Size", "Brightness", "Avg. Red", "Redness", "Norm.Brightness", "Norm.Avg. Red", "Norm.Redness"]
mean_data = change_col_names(mean_data, names, "mean")
std_data = change_col_names(std_data, names, "std")
count_data = count_data.rename(columns={"Size":"Count"})
count_data = count_data[["Count"]]

# Merge both dataframes:
combined_data = pd.merge(mean_data, std_data, how='left', on=["Run", "Plate", "mapped.pos", "Gene"])
combined_data = pd.merge(count_data, combined_data, how='left', on=["Run", "Plate", "mapped.pos", "Gene"])
print(combined_data)

                                Count  Size.mean  Brightness.mean  \
Run Plate mapped.pos Gene                                           
2   1     A10        FPS1           4     318.75        16.602205   
          A11        GDH3           4     367.00        17.454330   
          A12        RPL8B          4     332.00        16.041620   
          A13        BDH2           4     371.75        16.680479   
          A14        RNP1           4     367.50        16.445395   
...                               ...        ...              ...   
3   13    P21        NOT5           4     358.50       129.141042   
          P23        YOR008C-A      4     458.25       130.569227   
          P5         VPS15          4     403.25       126.947385   
          P7         FES1           4     474.25       120.776146   
          P9         MRPL36         4     346.25       125.984631   

                                Avg. Red.mean  Redness.mean  \
Run Plate mapped.pos Gene              

Finally, to not get biased by small colonies, the final data will be separated in 2 files based on cut-off = 100 on the average size of colonies:

In [12]:
cutOff = 100
below_data = combined_data[combined_data["Size.mean"] < cutOff]
above_data = combined_data[combined_data["Size.mean"] >= cutOff]
below_data.to_csv(join(INTERMEDIATE, "Betax05.12.19_ColonyData_belowCutOff.csv"))
above_data.to_csv(join(INTERMEDIATE, "Betax05.12.19_ColonyData_aboveCutOff.csv"))